In [1]:
from pathlib import Path


try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB and Path.cwd().name == "content":
    if not Path("quantum-jam-chirimbolo").exists():
        !git clone https://github.com/segusantos/quantum-jam-chirimbolo.git
        !pip install quantum-jam-chirimbolo/
    %cd quantum-jam-chirimbolo

Cloning into 'quantum-jam-chirimbolo'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 74 (delta 40), reused 27 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 110.65 KiB | 1.08 MiB/s, done.
Resolving deltas: 100% (40/40), done.
ERROR: You must give at least one requirement to install (see "pip help install")
/content/quantum-jam-chirimbolo


In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_bloch_vector
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, ParameterVector
import qiskit.qasm3
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService

ModuleNotFoundError: No module named 'qiskit'

In [ ]:
from src.test import test

test()